<p align="center" width="100%">
    <img width="66%" src="https://raw.githubusercontent.com/linukc/master_dlcourse/main/images/logo.png">
</p>

 # **[MIPT DL frameworks Autumn 2024](https://wiki.cogmodel.mipt.ru/s/mtai/doc/2024-nejrosetevye-frejmvorki-glubokogo-obucheniya-ZBGd69bxLd). Class 2: Segmentation**

# Самостоятельная работа

На основе библиотеки https://github.com/qubvel-org/segmentation_models.pytorch (или любой другой, выбранной вами) реализовать файнтюнинг модели семантической/инстанс сегментации на основе предоставленный весов (ImageNet, ...). В качестве набора данных можно использовать любой датасет, домен которого отличается от предоставленных весов. Обязательно провести 3 эксперимента с вариациями https://smp.readthedocs.io/en/latest/losses.html и 2 эксперимента с https://smp.readthedocs.io/en/latest/encoders.html. LR - ReduceLROnPlateau с warmup.

В качестве отчета предоставляется репозиторий с кодом, README.md с общими комментариями и pdf лог обучения (wandb, ...). Обязательно наличие метрик сегмемнтации на валидационной/тестовой выборке до/после обучения (минимум IoU).

https://smp.readthedocs.io/en/latest/quickstart.html

Оценка:

1 балл - задание полностью соответствует критериям

In [1]:
!pip install segmentation-models-pytorch

import torch
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.4 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=345273064e1840552192ecffacc6595b860a90ff6deadcc99d87f139702712c4
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=c015ddc761fe6e63ea31c1d85ed9678ad22b12eac741815daa49c1006025b29d
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!wget -q https://github.com/alexgkendall/SegNet-Tutorial/archive/master.zip -O camvid.zip
!unzip -q camvid.zip -d camvid_data

image_dir = "./camvid_data/SegNet-Tutorial-master/CamVid/train/"
mask_dir = "./camvid_data/SegNet-Tutorial-master/CamVid/trainannot/"

val_image_dir = "./camvid_data/SegNet-Tutorial-master/CamVid/val/"
val_mask_dir = "./camvid_data/SegNet-Tutorial-master/CamVid/valannot/"

train_images = sorted([os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith('.png')])
train_masks = sorted([os.path.join(mask_dir, msk) for msk in os.listdir(mask_dir) if msk.endswith('.png')])

val_images = sorted([os.path.join(val_image_dir, img) for img in os.listdir(val_image_dir) if img.endswith('.png')])
val_masks = sorted([os.path.join(val_mask_dir, msk) for msk in os.listdir(val_mask_dir) if msk.endswith('.png')])

In [10]:
class CamVidDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)

        # Add padding to make dimensions divisible by 32
        h, w, _ = image.shape
        pad_h = (32 - h % 32) % 32
        pad_w = (32 - w % 32) % 32

        image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=0)
        mask = cv2.copyMakeBorder(mask, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=0)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        image = transforms.ToTensor()(image)
        mask = torch.tensor(mask, dtype=torch.long)
        return image, mask

def calculate_iou(predicted, target, num_classes):
    predicted = torch.argmax(predicted, dim=1)
    iou_per_class = []

    for cls in range(num_classes):
        intersection = torch.sum((predicted == cls) & (target == cls))
        union = torch.sum((predicted == cls) | (target == cls))

        if union == 0:
            iou_per_class.append(float('nan'))  # Пропускаем класс, если он отсутствует
        else:
            iou_per_class.append((intersection / union).item())

    return np.nanmean(iou_per_class)

def get_transforms():
    return None

train_dataset = CamVidDataset(train_images, train_masks, transform=get_transforms())
val_dataset = CamVidDataset(val_images, val_masks, transform=get_transforms())

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",    # Use ImageNet weights
    in_channels=3,
    classes=32                      # CamVid has 32 classes
).to(device)

loss_fn = smp.losses.DiceLoss(mode='multiclass')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

from torch.optim.lr_scheduler import OneCycleLR

scheduler = OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=10)

def train_one_epoch(model, loader, optimizer, loss_fn, scheduler_flag=True):
    model.train()
    total_loss = 0
    for images, masks in tqdm(loader):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        if scheduler_flag:
            scheduler.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def validate(model, loader, loss_fn):
    model.eval()
    total_loss = 0
    total_iou = 0

    with torch.no_grad():
        for images, masks in tqdm(loader):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, masks)

            total_loss += loss.item()
            total_iou += calculate_iou(outputs, masks, num_classes=32)

    avg_loss = total_loss / len(loader)
    avg_iou = total_iou / len(loader)

    return avg_loss, avg_iou

In [18]:
loss_functions = {
    "DiceLoss": smp.losses.DiceLoss(mode='multiclass'),
    "JaccardLoss": smp.losses.JaccardLoss(mode='multiclass'),
    "FocalLoss": smp.losses.FocalLoss(mode='multiclass')
}

encoders = ["resnet34", "densenet121"]

results = []

for encoder in encoders:
    for loss_name, loss_fn in loss_functions.items():
        print(f"Experiment with Encoder: {encoder}, Loss: {loss_name}")

        model = smp.Unet(
            encoder_name=encoder,
            encoder_weights="imagenet",
            in_channels=3,
            classes=32
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        scheduler = OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=10)

        for epoch in range(10):
            print(f"Epoch {epoch + 1}/10")
            train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, scheduler)
            val_loss, val_iou = validate(model, val_loader, loss_fn)

            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val IoU: {val_iou:.4f}")

        results.append({
            "Encoder": encoder,
            "Loss Function": loss_name,
            "Val Loss": val_loss,
            "Val IoU": val_iou
        })

Experiment with Encoder: resnet34, Loss: DiceLoss
Epoch 1/10


  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

Train Loss: 0.3379 | Val Loss: 0.3004 | Val IoU: 0.1323
Epoch 2/10


  0%|          | 0/92 [00:00<?, ?it/s]

...

  0%|          | 0/26 [00:00<?, ?it/s]

Train Loss: 0.1213 | Val Loss: 0.1479 | Val IoU: 0.4790
Epoch 10/10


  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

Train Loss: 0.1176 | Val Loss: 0.1517 | Val IoU: 0.4725


In [19]:
import pandas as pd

results_df = pd.DataFrame(results)
print(results_df)
results_df.to_csv("experiment_results.csv", index=False)

       Encoder Loss Function  Val Loss   Val IoU
0     resnet34      DiceLoss  0.111093  0.620937
1     resnet34   JaccardLoss  0.148295  0.607671
2     resnet34     FocalLoss  0.156535  0.441159
3  densenet121      DiceLoss  0.105749  0.630606
4  densenet121   JaccardLoss  0.151662  0.600078
5  densenet121     FocalLoss  0.151727  0.472506


In [22]:
encoders = ["resnet34", "densenet121"]
loss_functions = {
    "DiceLoss": smp.losses.DiceLoss(mode='multiclass'),
}

evaluation_results = []

for encoder in encoders:
    for loss_name, loss_fn in loss_functions.items():
        print(f"Experiment with Encoder: {encoder}, Loss: {loss_name}")

        model = smp.Unet(
            encoder_name=encoder,
            encoder_weights="imagenet",
            in_channels=3,
            classes=32
        ).to(device)

        val_loss, val_iou = 0, 0
        with torch.no_grad():
            for images, masks in tqdm(val_loader):
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                val_loss += loss_fn(outputs, masks).item()
                val_iou += calculate_iou(outputs, masks, num_classes=32)

        val_loss /= len(val_loader)
        val_iou /= len(val_loader)

        print(f"Val Loss: {val_loss:.4f} | Val IoU: {val_iou:.4f}")

        evaluation_results.append({
            "Encoder": encoder,
            "Loss Function": loss_name,
            "Val Loss": val_loss,
            "Val IoU": val_iou
        })

evaluation_results_df = pd.DataFrame(evaluation_results)
print(evaluation_results_df)
evaluation_results_df.to_csv("evaluation_results.csv", index=False)

Experiment with Encoder: resnet34, Loss: DiceLoss


  0%|          | 0/26 [00:00<?, ?it/s]

Val Loss: 0.3620 | Val IoU: 0.0101
Experiment with Encoder: densenet121, Loss: DiceLoss


  0%|          | 0/26 [00:00<?, ?it/s]

Val Loss: 0.3603 | Val IoU: 0.0106
       Encoder Loss Function  Val Loss   Val IoU
0     resnet34      DiceLoss  0.361983  0.010100
1  densenet121      DiceLoss  0.360312  0.010632
